In [ ]:
import os
import re
import sys
import glob
import pickle
import numpy as np
from scipy.fft import fft, fftfreq
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
import seaborn as sns

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from dlml.data import load_data_files, load_data_areas

In [ ]:
def make_axes(rows, cols, x_offset, y_offset, x_space, y_space, squeeze=True):
    w = (1 - np.sum(x_offset) - x_space * (cols - 1)) / cols
    h = (1 - np.sum(y_offset) - y_space * (rows - 1)) / rows
    
    ax = [[plt.axes([x_offset[0] + (w + x_space) * j,
                     y_offset[0] + (h + y_space) * i,
                     w, h]) for j in range(cols)] for i in range(rows-1, -1, -1)]
    
    for row in ax:
        for a in row:
            for side in 'right','top':
                a.spines[side].set_visible(False)

    if squeeze:
        if rows == 1 and cols == 1:
            return ax[0][0]
        if rows == 1:
            return ax[0]
        if cols == 1:
            return [a[0] for a in ax]
        
    return ax

In [ ]:
def plot_correlations(R, p, R_ctrl, p_ctrl, edges, idx, ax, sort_freq=1.0,
                      vmin=None, vmax=None, legend_bbox=[0.4, -0.05]):
    if p is not None:
        R = R.copy()
        R[p > 0.05] = np.nan
    if R_ctrl is not None and p_ctrl is not None:
        R_ctrl = R_ctrl.copy()
        R_ctrl[p_ctrl > 0.05] = np.nan
    rows, cols = ax.shape
    if rows != len(idx):
        raise Exception('Number of rows of ax does not match len(idx)')
    R_mean = [np.nanmean(R[jdx], axis=0) for jdx in idx]
    R_abs_mean = [np.mean(np.abs(r), axis=1) for r in R_mean]
    if R_ctrl is not None:
        R_ctrl_mean = [np.nanmean(R_ctrl[jdx], axis=0) for jdx in idx]
        R_ctrl_abs_mean = [np.mean(np.abs(r), axis=1) for r in R_ctrl_mean]
    else:
        R_ctrl_mean = [None for _ in range(rows)]
        R_ctrl_abs_mean = None
    if np.isscalar(sort_freq):
        sort_freq += np.zeros(rows)
    edge = np.array([np.abs(edges - freq).argmin() for freq in sort_freq])
    for i in range(rows):
        kdx = np.argsort(R_mean[i][edge[i],:])
        R_mean[i] = R_mean[i][:,kdx]
        if R_ctrl is not None:
            R_ctrl_mean[i] = R_ctrl_mean[i][:,kdx]

    make_symmetric = False
    if vmin is None:
        vmin = min([r.min() for r in R_mean])
        make_symmetric = True
    if vmax is None:
        vmax = max([r.max() for r in R_mean])
        if make_symmetric:
            if vmax > np.abs(vmin):
                vmin = -vmax
            else:
                vmax = -vmin
    print(f'Color bar bounds: ({vmin:.2f},{vmax:.2f}).')
    ticks = np.linspace(vmin, vmax, 7)
    ticklabels = [f'{tick:.2f}' for tick in ticks]

    cmap = plt.get_cmap('bwr')
    y = edges[:-1] + np.diff(edges) / 2
    for i in range(rows):
        for j,R in enumerate((R_mean[i], R_ctrl_mean[i])):
            if R is not None:
                x = np.arange(R.shape[-1])
                im = ax[i][j].pcolormesh(x, y, R, vmin=vmin, vmax=vmax, shading='auto', cmap=cmap)
                for side in 'right','top':
                    ax[i][j].spines[side].set_visible(False)
                ax[i][j].set_xticks(np.linspace(0, x[-1], 3, dtype=np.int32))
        if cols > 1:
            cbar = plt.colorbar(im, fraction=0.1, shrink=1, aspect=20, label='Correlation',
                                orientation='vertical', ax=ax[i][1], ticks=ticks)
            cbar.ax.set_yticklabels(ticklabels, fontsize=fontsize-1)

            ax[i][-1].plot(R_abs_mean[i], y, 'r', lw=1, label='Tr.')
            if R_ctrl_abs_mean is not None:
                ax[i][-1].plot(R_ctrl_abs_mean[i], y, 'g--', lw=1, label='Untr.')
                ax[i][-1].plot(R_abs_mean[i] - R_ctrl_abs_mean[i], y, 'k', lw=1, label='Diff.')
            ax[i][-1].legend(loc='lower left', bbox_to_anchor=legend_bbox,
                             frameon=False, fontsize=fontsize-1)

    for i in range(rows):
        for j in range(cols):
            ax[i][j].set_ylim(edges[[0,-2]])
            ax[i][j].set_yscale('log')

    return vmin, vmax

In [ ]:
def plot_layer_output_hist(Y, group_index, N_bins, cols=8, w=2, h=1.5, cmap=None, ax=None, labels=None):
    N_trials, N_samples, N_filters = Y.shape
    N_groups = len(group_index)
    N = np.zeros((N_filters, N_groups, N_bins))
    edges = np.zeros((N_filters, N_groups, N_bins+1))
    for i in range(N_filters):
        for j,jdx in enumerate(group_index):
            N[i,j,:],edges[i,j,:] = np.histogram(Y[jdx, :, i], N_bins)

    if cmap is None:
        cmap = plt.get_cmap('tab10', N_groups)
    if ax is None:
        rows = N_filters // cols
        fig,ax = plt.subplots(rows, cols, figsize=(w*cols, h*rows))
    else:
        fig = None
        N_filters = ax.size
    ax = ax.flatten()
    for i in range(N_filters):
        for j in range(N_groups):
            de = np.diff(edges[i, j, :])[0]
            col = np.max([[0,0,0], cmap(j)[:3] - 1/3 * np.ones(3)], axis=0)
            ax[i].bar(edges[i, j, :-1], N[i, j, :], width=de*0.8, align='edge',
                     facecolor=cmap(j), edgecolor=col, linewidth=0.5, alpha=0.85)
        xlim = [edges[i, :, 2:-3].min(), edges[i, j, 2:-3].max()]
        ylim = ax[i].get_ylim()
#         ax[i].set_xlim(xlim)
#         ax[i].set_xticks(xlim)
        if labels is not None:
            ax[i].text(xlim[0] - 0.1 * np.diff(xlim), ylim[1],
                       labels[i], fontsize=fontsize-1, verticalalignment='top',
                       horizontalalignment='left')
        ax[i].set_xticklabels([])
        ax[i].set_yticks(ax[i].get_ylim())
        ax[i].set_yticklabels([])
        for side in 'right','top':
            ax[i].spines[side].set_visible(False)
    if fig is not None:
        fig.tight_layout()
    return fig,ax

In [ ]:
X, y, Xf = {}, {}, {}
group_index, n_mom_groups = {}, {}

In [ ]:
# data_file = 'traces_hist_spectra.npz'
data_file = 'traces_hist_spectra_comp_grid.npz'
force = False
if not os.path.isfile(data_file) or force:
    if data_file == 'traces_hist_spectra.npz':
        set_name = 'test'
    else:
        set_name = 'training'
    data_dir = '../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/var_H_area_1'
    data_files = sorted(glob.glob(data_dir + os.path.sep + f'*_{set_name}_set.h5'))
    var_names = ['Vd_bus3']
    generators_areas_map = [['G02', 'G03'], ['G04', 'G05', 'G06', 'G07'], ['G08', 'G09', 'G10'], ['G01']]
    generators_Pnom = {'G01': 10e9, 'G02': 700e6, 'G03': 800e6, 'G04': 800e6, 'G05': 300e6,
                       'G06': 800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6}
    area_measure = 'momentum'
    ret = load_data_areas({set_name: data_files}, var_names,
                          generators_areas_map[:1],
                          generators_Pnom,
                          area_measure,
                          trial_dur=60,
                          max_block_size=1000,
                          use_tf=False,
                          add_omega_ref=True,
                          use_fft=False)
    t = ret[0]
    X_raw = ret[1][set_name]
    y[set_name] = ret[2][set_name]
    group_index[set_name] = [np.where(y[set_name] == mom)[0] for mom in np.unique(y[set_name])]
    n_mom_groups[set_name] = len(group_index[set_name])
    X_mean, X_std = X_raw.mean(axis=(1,2)), X_raw.std(axis=(1,2))
    X[set_name] = (X_raw - X_mean) / X_std
    X[set_name] = X[set_name].squeeze()
    y[set_name] = y[set_name].squeeze()

    dt = np.diff(t[:2])[0]
    N_samples = t.size
    Xf[set_name] = fft(X[set_name])
    Xf[set_name] = 2.0 / N_samples * np.abs(Xf[set_name][:, :N_samples//2])
    F = fftfreq(N_samples, dt)[:N_samples//2]


    data_dir = '../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/low_momentum_test_var_G1_G2'
    data_files = sorted(glob.glob(data_dir + os.path.sep + '*.h5'))
    idx = list(map(lambda f: len(re.findall('_[0-9]*.h5', f)) > 0, data_files))
    data_files = [data_files[j] for j,i in enumerate(idx) if i]
    N = len(np.unique([int(os.path.splitext(re.findall('_[0-9]*.h5', f)[0])[0][1:]) for f in data_files]))
    M = len(data_files) // N
    group_index['var_G2_G3'] = np.reshape(np.arange(len(data_files)), (M,N))

    ret = load_data_files(data_files,
                          var_names,
                          generators_areas_map[:1],
                          generators_Pnom,
                          'momentum')

    X_raw = ret[1][:, :, :-1]
    X['var_G2_G3'] = np.zeros(X_raw.shape)
    X['var_G2_G3'] = (X_raw - X_mean) / X_std
    y['var_G2_G3'] = ret[2]
    X['var_G2_G3'] = X['var_G2_G3'].squeeze()
    y['var_G2_G3'] = y['var_G2_G3'].squeeze()
    Xf['var_G2_G3'] = fft(X['var_G2_G3'])
    Xf['var_G2_G3'] = 2.0 / N_samples * np.abs(Xf['var_G2_G3'][:, :N_samples//2])
    n_mom_groups['var_G2_G3'] = len(group_index['var_G2_G3'])
    
    np.savez_compressed(data_file, t=t, X=X, y=y, Xf=Xf, F=F,
                        group_index=group_index, n_mom_groups=n_mom_groups)
else:
    if data_file == 'traces_hist_spectra.npz':
        set_name = 'test'
    else:
        set_name = 'training'
    data = np.load(data_file, allow_pickle=True)
    t = data['t']
    X = data['X'].item()
    y = data['y'].item()
    F = data['F']
    Xf = data['Xf'].item()
    group_index = data['group_index'].item()
    n_mom_groups = data['n_mom_groups'].item()

## Figure 2

In [ ]:
fig,ax = plt.subplot_mosaic(
    '''
    AAAAAAAA
    BBBBBCCC
    DDDDEEEE
    ''',
    figsize=(3.5,4)
)
cmap = plt.get_cmap('Set1')
N_div_cmap = 10
div_cmap = plt.get_cmap('bwr', N_div_cmap)

###########################################
momentum = lambda H, S, fn: 2 * H@S / fn * 1e-3

N = 11, 11
h_G2_0, h_G3_0 = 4.33, 4.47
h_G2 = h_G2_0 + np.linspace(-1, 1, N[0])
h_G3 = h_G3_0 + np.linspace(-1, 1, N[1])
H_G2, H_G3 = np.meshgrid(*[h_G2, h_G3])

M = np.zeros(N)
S = np.array([700, 800])
fn = 60
for i in range(N[0]):
    for j in range(N[1]):
        H = np.array([H_G2[i,j], H_G3[i,j]])
        M[i,j] = momentum(H, S, fn)
        
gray_cmap = plt.get_cmap('gray')
cont = ax['A'].contourf(H_G2, H_G3, M, levels=100, cmap=gray_cmap, zorder=-1)
cbar = plt.colorbar(cont, ax=ax['A'])
white = [1,1,1]
magenta = [1,0,1]
green = [0,1,0]
yellow = [1,1,0]
blue = [0,.5,1]
red = [1,.333,.333]
orange = [1, .5, 0]
gray = [.6, .6, .6]
zord = 0
for i in range(2):
    for j in range(2):
        ax['A'].plot(H_G2[i,j], H_G3[i,j], 's', markersize=4, lw=1, color=div_cmap(i*2+j), zorder=zord)
        ax['A'].plot(H_G2[-1-i,-1-j], H_G3[-1-i,-1-j], 's', markersize=4, lw=1,
                     color=div_cmap(N_div_cmap-1-i*2-j), zorder=zord)
        zord += 2
for i in range(0, N[0], 2):
    ax['A'].plot(H_G2[i,i], H_G3[i,i], 'o', color=cmap(i//2), markersize=3, lw=1, zorder=zord)
    zord += 1
ax['A'].plot(H_G2[:2,:2].mean(), H_G3[:2,:2].mean(), 'x', color=magenta, markersize=4,
             markeredgewidth=1, zorder=zord)
zord += 1
ax['A'].plot(H_G2[-2:,-2:].mean(), H_G3[-2:,-2:].mean(), 'x', color=magenta, markersize=4,
             markeredgewidth=1, zorder=zord)
zord += 1
ax['A'].scatter(H_G2[::2, ::2], H_G3[::2, ::2], s=5, c='w', edgecolors='k', lw=0.5, marker='o', zorder=zord)
ax['A'].set_xlabel(r'$H_{G_2}$ [s]')
ax['A'].set_ylabel(r'$H_{G_3}$ [s]')
ax['A'].set_xlim([h_G2_0 - 1.1, h_G2_0 + 1.1])
ax['A'].set_ylim([h_G3_0 - 1.1, h_G3_0 + 1.1])
ax['A'].set_xticks([h_G2_0 - 1, h_G2_0, h_G2_0 + 1])
ax['A'].set_yticks([h_G3_0 - 1, h_G3_0, h_G3_0 + 1])
cbar.set_label(r'Momentum [GW$\cdot$s$^2$]')
cbar.set_ticks(np.r_[0.17 : 0.28 : 0.02])
###########################################

tend = 10
jdx, = np.where(t < tend)
for i,idx in enumerate(group_index[set_name]):
    n,edges = np.histogram(X[set_name][idx,:], bins=50, density=True)
    ax['B'].plot(t[jdx], X[set_name][idx[0]+1, jdx], lw=1, color=cmap(i))
    ax['C'].plot(n, edges[1:], lw=1, color=cmap(i))
    ax['D'].plot(F, 20 * np.log10(Xf[set_name][idx,:].mean(axis=0)), lw=1,
               color=cmap(i), label=r'{:.3f} GW$\cdot$s$^2$'.format(y[set_name][idx[0]+1]))

for i,idx in enumerate(group_index['var_G2_G3']):
    if i < 4:
        ax['E'].plot(F, 20 * np.log10(Xf['var_G2_G3'][idx,:].mean(axis=0)), lw=1,
                   color=div_cmap(i))
    else:
        ax['E'].plot(F, 20 * np.log10(Xf['var_G2_G3'][idx,:].mean(axis=0)), lw=1,
                   color=div_cmap(N_div_cmap - (i - 3)))

for key in 'BC':
    ax[key].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
for key in 'DE':
    ax[key].grid(which='major', axis='x', lw=0.5, ls=':', color=[.6,.6,.6])
for a in ax.values():
    for side in 'right','top':
        a.spines[side].set_visible(False)
for key in 'BC':
    ax[key].set_ylim([-3.5, 3.5])
    ax[key].set_yticks(np.r_[-3 : 4 : 1.5])
for key in 'DE':
    ax[key].set_ylim([-55, -10])
    ax[key].set_yticks(np.r_[-50 : -9 : 10])
    ax[key].set_xscale('log')
    ax[key].set_xlabel('Frequency [Hz]')
    ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
    ax[key].set_xlim(ticks[[0,-1]] + np.array([0,1]))
    ax[key].xaxis.set_major_locator(FixedLocator(ticks))
    ax[key].xaxis.set_minor_locator(NullLocator())
    ax[key].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
ax['C'].set_yticklabels([])
ax['E'].set_yticklabels([])

ax['B'].set_xlabel('Time [s]')
ax['B'].set_ylabel('Norm. V')
ax['C'].set_xlabel('Distr.')
ax['D'].set_ylabel('Power [dB]')

ticks = np.r_[0 : 10.5 : 2]
ax['B'].set_xlim(ticks[[0,-1]])
ax['B'].xaxis.set_major_locator(FixedLocator(ticks))
ax['B'].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))

ticks = np.r_[0 : 0.51 : 0.25]
ax['C'].set_xlim(ticks[[0,-1]])
ax['C'].xaxis.set_major_locator(FixedLocator(ticks))
ax['C'].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))

trans = mtransforms.ScaledTranslation(-0.45, -0.05, fig.dpi_scale_trans)
for label in 'ABD':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=10, va='bottom')
trans = mtransforms.ScaledTranslation(-0.15, -0.05, fig.dpi_scale_trans)
for label in 'CE':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=10, va='bottom')

fig.tight_layout(pad=0)
plt.savefig('traces_hist_spectra_comp_grid.pdf')

## Figure 3

In [ ]:
experiment_ID = '98475b819ecb4d569646d7e1467d7c9c'
# experiment_ID = 'ed79ae2784274401a9dba5f5ccee98d8'
experiments_path = '../experiments/neural_network/'
history = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'history.pkl'), 'rb'))
test_results = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'test_results.pkl'), 'rb'))
N_bands = 60
filter_order = 6
if experiment_ID == '98475b819ecb4d569646d7e1467d7c9c':
    N_trials = 4000
elif experiment_ID == 'ed79ae2784274401a9dba5f5ccee98d8':
    N_trials = 1000
else:
    raise Exception(f'Unknown number of trials for experiment {experiment_ID[:6]}')
correlations_file = f'correlations_{experiment_ID[:6]}_{N_bands}-bands_64-filters_' + \
    f'36-neurons_{N_trials}-trials_{filter_order}-butter_Vd_bus3_pool_1_3.npz'
correlations = np.load(os.path.join(experiments_path, experiment_ID, correlations_file))
for key in correlations.files:
    exec(f'{key} = correlations["{key}"]')
corr_idx = idx
corr_edges = edges

In [ ]:
key = 'var_G2_G3'

fig = plt.figure(figsize=(3.5 * 2, 4))

offset = 0.07, 0.1
border = 0.01, 0.02
space = ((0.05, 0.1, 0.1), (0.075, 0.075, 0.13)), 0.17
rows = 2
h = (1 - offset[1] - border[1] - space[1] * (rows-1)) / rows
w_rel = [
    [3, 2, 3, 2],
    [2, 3, 3, 2]
]

w_rel_sum = np.sum(w_rel, axis=1)
cols = list(map(len, w_rel))
w_total = [1 - offset[0] - border[0] - np.sum(sp) for sp in space[0]]
w = []
for i in range(rows):
    w.append([])
    for j in range(cols[i]):
        w[-1].append(w_total[i] * w_rel[i][j] / w_rel_sum[i])

labels = ['ABCD','EFGH']
ax = {}
for i in range(rows):
    for j in range(cols[i]):
        ax[labels[i][j]] = fig.add_axes([offset[0] + np.sum(space[0][i][:j]) + np.sum(w[i][:j]),
                                         1 - border[1] - h * (i+1) - space[1] * i,
                                         w[i][j],
                                         h])

# ############# Panel A #############
cmap = plt.get_cmap('tab10')
green, magenta = cmap(2), cmap(6)
jdx, = np.where(t < tend)
ax['A'].plot(t[jdx], X[key][:5, jdx].T, color=green, lw=0.5)
ax['A'].plot(t[jdx], X[key][-10:-5, jdx].T, color=magenta, lw=0.5)

# ############# Panel B #############
idx = np.concatenate(group_index[key][:4])
n,edges = np.histogram(X[key][idx,:], bins=50, density=True)
ax['B'].plot(n, edges[1:], lw=1, color=green)
m = Xf[key][idx,:].mean(axis=0)
ci = 1.96 * Xf[key][idx,:].std(axis=0) / np.sqrt(idx.size)
ax['E'].plot(20 * np.log10(m), F, color=green, lw=1)
idx = np.concatenate(group_index[key][4:])
n,edges = np.histogram(X[key][idx,:], bins=50, density=True)
ax['B'].plot(n, edges[1:], lw=1, color=magenta)
m = Xf[key][idx,:].mean(axis=0)
ci = 1.96 * Xf[key][idx,:].std(axis=0) / np.sqrt(idx.size)
ax['E'].plot(20 * np.log10(m), F, color=magenta, lw=1)
ax['B'].set_xlim([0, 0.5])
ticks = np.r_[0 : 0.51 : 0.25]
ax['B'].xaxis.set_major_locator(FixedLocator(ticks))
ax['B'].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
ax['B'].set_yticklabels([])

# ############# Panel C #############
ax['C'].plot(history['loss'], 'k', lw=1, label='Training')
ax['C'].plot(history['val_loss'], 'r', lw=1, label='Validation')
ax['C'].legend(loc='upper right', frameon=False, fontsize=fontsize)

# ############# Panel D #############
target_values = np.unique(exact_momentum)
df = pd.DataFrame(data={'exact': exact_momentum, 'pred': np.concatenate(pred_momentum)})
df_ctrl = pd.DataFrame(data={'exact': exact_momentum, 'pred': np.concatenate(pred_momentum_ctrl)})
# sns.violinplot(x='exact', y='pred', data=df_ctrl, cut=0, inner='quartile',
#                palette='gray', ax=ax['D'], linewidth=0.5)
sns.violinplot(x='exact', y='pred', data=df, cut=0, inner='quartile',
               palette=[green, magenta], ax=ax['D'], linewidth=0.5)
ax['D'].xaxis.set_major_locator(FixedLocator([0, 1]))
ax['D'].xaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in target_values]))
ax['D'].yaxis.set_major_locator(FixedLocator(target_values))
ax['D'].yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in target_values]))

# ############# Panels F, G ########
plot_correlations(R, p, R_ctrl, p_ctrl, corr_edges,
                  [np.concatenate(corr_idx)], sort_freq=[1.1],
                  ax=np.array([[ax['F'], ax['G'], ax['H']]]))
ax['F'].set_title('Trained network', fontsize=fontsize+1)
ax['G'].set_title('Untrained network', fontsize=fontsize+1)

for label in 'AB':
    ax[label].set_ylim([-3.5, 3.5])
    ax[label].set_yticks(np.r_[-3 : 3.1])

for label in 'ABE':
    ax[label].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])

for a in ax.values():
    for side in 'right','top':
        a.spines[side].set_visible(False)

for label in 'EFG':
    ax[label].set_yscale('log')
    ax[label].set_ylim([0.05, 20])

# ax['E'].invert_xaxis()
# ax['E'].yaxis.tick_right()
# ax['E'].spines['right'].set_visible(True)
# ax['E'].spines['left'].set_visible(False)
ax['E'].set_xlim([-10, -55])
ax['E'].set_xticks(np.r_[-50 : -9 : 10])

for label in 'EFGH':
    ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
    ax[label].set_ylim(ticks[[0,-1]])
    ax[label].yaxis.set_major_locator(FixedLocator(ticks))
    ax[label].yaxis.set_minor_locator(NullLocator())
    if label == 'F' or True:
        ax[label].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))

ax['A'].set_xlabel('Time [s]')
ax['A'].set_ylabel('Norm. V')
ax['B'].set_xlabel('Distribution')
ax['C'].set_xlabel('Epoch')
ax['C'].set_ylabel('Loss')
ax['D'].set_xlabel(r'Exact M [GW$\cdot$s$^2$]')
ax['D'].set_ylabel(r'Predicted M [GW$\cdot$s$^2$]')
ax['E'].set_xlabel('Power [dB]')
ax['E'].set_ylabel('Frequency [Hz]')
ax['F'].set_xlabel('Filter #')
ax['G'].set_xlabel('Filter #')
ax['H'].set_xlabel('Correlation')
ax['F'].set_xticklabels([1, 32, 64])
ax['G'].set_xticklabels([1, 32, 64])

trans = mtransforms.ScaledTranslation(-0.3, -0.05, fig.dpi_scale_trans)
for label in 'ABCD':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=10, va='bottom')
trans = mtransforms.ScaledTranslation(-0.4, -0.05, fig.dpi_scale_trans)
for label in 'EFGH':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=10, va='bottom')

plt.savefig(f'low_high_momentum_{experiment_ID[:6]}.pdf')

### Figure 4

In [ ]:
data = np.load(os.path.join(experiments_path, experiment_ID, 'stopband_momentum_estimation.npz'))
for key in data.files:
    exec(f'{key} = data["{key}"]')
N_bands = len(bands)
Xfm = np.zeros((len(group_index), F.size))
Xfci = np.zeros((len(group_index), F.size))
for i,idx in enumerate(group_index):
    Xfm[i] = Xf[idx].mean(axis=0)
    Xfci[i] = 1.96 * Xf[idx].std(axis=0) / np.sqrt(idx.size)

In [ ]:
fig,ax = plt.subplot_mosaic(
    '''
    AAAA.
    BBBB.
    ''',
    figsize=(3.5,4)
)

cmap2 = plt.get_cmap('Set1')
y_m = [y[idx].mean() for idx in group_index]
y_s = [y[idx].std() for idx in group_index]
y_pred_m = np.array([[pred[jdx].mean() for jdx in group_index] for pred in y_pred])
y_pred_s = np.array([[pred[jdx].std() for jdx in group_index] for pred in y_pred])
add_band = False
last_band = N_bands + 1 if add_band else N_bands + 1
for i in range(last_band):
    if i == 0:
        lbl = 'Broadband'
    else:
        lbl = f'[{bands[i-1][0]}-{bands[i-1][1]:g}] Hz'
    ax['A'].plot(y_m, y_pred_m[i], color=cmap2(i), lw=1, label=lbl)
    for j in range(len(group_index)):
        ax['A'].plot(y_m[j] + np.zeros(2),
                   y_pred_m[i,j] + y_pred_s[i,j] * np.array([-1,1]),
                   color=cmap2(i), lw=1)
        ax['A'].plot(y_m[j] + y_s[j] * np.array([-1,1]),
                   y_pred_m[i,j] + np.zeros(2),
                   color=cmap2(i), lw=1)
        ax['A'].plot(y_m[j], y_pred_m[i,j], 'o', color=cmap2(i),
                   markerfacecolor='w', markersize=4, markeredgewidth=1)
for side in 'right','top':
    ax['A'].spines[side].set_visible(False)
ax['A'].legend(loc='center left', bbox_to_anchor=[0.875, 0.5], frameon=False, fontsize=fontsize-1)
ax['A'].set_xlabel(r'Exact M [GW$\cdot$s$^2$]')
ax['A'].set_ylabel(r'Predicted M [GW$\cdot$s$^2$]')
ax['A'].set_xlim(y_m + np.diff(y_m) * np.array([-1/10, 1/5]))
ax['A'].set_ylim(y_m + np.diff(y_m) / 3 * np.array([-1,1]))
ax['A'].xaxis.set_major_locator(FixedLocator(y_m))
ax['A'].xaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in y_m]))
ax['A'].yaxis.set_major_locator(FixedLocator(y_m))
ax['A'].yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in y_m]))

axr = ax['B'].twinx()
for i,(m,ci,col) in enumerate(zip(Xfm, Xfci, (green,magenta))):
    ax['B'].plot(F, 20*np.log10(m), color=col,
                 label=r'M = {:.2f} GW$\cdot$s$^2$'.format(y[group_index[i]].mean()))
ax['B'].legend(loc='lower left', frameon=False, fontsize=fontsize-1)
axr.plot(ax['B'].get_xlim(), scores[0] + np.zeros(2), '--', color=cmap2(0), lw=2)
for i,band in enumerate(bands):
    if i >= last_band-1:
        break
    axr.axvline(band[0], color=[.6,.6,.6], ls=':', lw=0.5)
    axr.plot(band, scores[i+1] + np.zeros(2), color=cmap2(i+1), lw=2)
ax['B'].set_xlabel('Frequency [Hz]')
ax['B'].set_ylabel('Power [dB]')
axr.set_ylabel(r'R$^2$ score')
ax['B'].set_xscale('log')

ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
ax['B'].set_xlim(ticks[[0,-1]] + np.array([0,1]))
ax['B'].xaxis.set_major_locator(FixedLocator(ticks))
ax['B'].xaxis.set_minor_locator(NullLocator())
ax['B'].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
axr.set_ylim((-0.25, 1.05))
axr.set_yticks(np.r_[-0.2 : 1.05 : 0.2])

trans = mtransforms.ScaledTranslation(-0.45, -0.05, fig.dpi_scale_trans)
for label in 'AB':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=10, va='bottom')

fig.tight_layout(pad=0.2)
if add_band:
    fig.savefig(f'stopband_momentum_estimation_{experiment_ID}_{last_band}.pdf')
else:
    fig.savefig(f'stopband_momentum_estimation_{experiment_ID[:6]}.pdf')

### Figure 5

In [ ]:
data = np.load(os.path.join(experiments_path, experiment_ID, 'layer_outputs.npz'))
multi_output = {}
for key in data.files:
    if 'multi_output' in key:
        i = int(key.split('_')[-1])
        multi_output[i] = data[key]
    else:
        exec(f'{key} = data["{key}"]')

In [ ]:
w, h = 3/2.54, 1.5/2.54
rows, cols = len(multi_output), 5
fig,ax = plt.subplots(rows, cols, figsize=(cols*w, rows*h))
for i in range(rows):
    dst = np.abs(np.diff(np.array([multi_output[i][idx,:,:].mean(axis=(0,1)) \
                                   for idx in group_index]), axis=0)).squeeze()
    idx = np.argsort(dst)[::-1]
    tmp = multi_output[i][:, :, idx]
    _,_ = plot_layer_output_hist(tmp, group_index, N_bins=16, ax=ax[i],
                                 labels=[f'#{j}' for j in idx],
                                 cmap = lambda i: [green, magenta][i])
    ss = layer_names[i+1].split('_')
    lbl = ' '.join([ss[2].capitalize(), ss[4]])
    ax[i,0].set_ylabel(lbl)
fig.tight_layout(pad=0.5)
fig.savefig(f'layer_outputs_distr_{experiment_ID[:6]}.pdf')

### Figure 6

In [ ]:
experiment_IDs = '98475b819ecb4d569646d7e1467d7c9c', '302a21340f354ac2949184be98d8e907'
experiments_path = '../experiments/neural_network/'

data, correlations = {}, {}
N_bands = 60
filter_order = 6
for experiment_ID,N_trials in zip(experiment_IDs, (4000,8000)):
    key = experiment_ID[:6]
    tmp = np.load(os.path.join(experiments_path, experiment_ID, f'variable_inertia_{key}.npz'),
                  allow_pickle=True)
    data[key] = {}
    for fname in tmp.files:
        try:
            exec(f'data["{key}"]["{fname}"] = tmp["{fname}"].item()')
        except:
            exec(f'data["{key}"]["{fname}"] = tmp["{fname}"]')
    correlations_file = f'correlations_{key}_{N_bands}-bands_64-filters_' + \
        f'36-neurons_{N_trials}-trials_{filter_order}-butter_Vd_bus3_pool_1_3.npz'
    correlations[key] = np.load(os.path.join(experiments_path, experiment_ID, correlations_file))

In [ ]:
fig,ax = plt.subplot_mosaic(
    '''
    AAABBBCC
    DDDEEEFF
    ''',
    figsize=(3.5 * 1.8, 3.5)
)

cmap_name = 'tab10'
cmap = plt.get_cmap(cmap_name)

################### Panels A and B ###################
key = experiment_IDs[0][:6]
ticks = {'A': np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20]),
         'B': np.array([0.4, 0.5, 0.7, 1, 1.5])}
for a in 'AB':
    n = 0
    for i,(k,v) in enumerate(data[key]['Xf'].items()):
        for j in range(data[key]['n_mom_groups'][k]):
            lbl = r'{:.3f} GW$\cdot$s$^2$'.format(data[key]['ym'][k][j])
            if k == 'var_G2_G3':
                lbl += ' (GEN)'
            elif k == 'var_Comp11':
                lbl += ' (COMP)'
            m = v[data[key]['group_index'][k][j], :].mean(axis=0)
            s = v[data[key]['group_index'][k][j], :].std(axis=0)
            ci = 1.96 * s / np.sqrt(data[key]['group_index'][k][j].size)
            ax[a].fill_between(data[key]['F'],
                               20*np.log10(m + ci),
                               20*np.log10(m - ci),
                               color=cmap(n), facecolor=cmap(n), edgecolor=cmap(n),
                               alpha=0.5, label=lbl)
            n += 1
    ax[a].set_xscale('log')
    ax[a].set_xlabel('Frequency [Hz]')
    for side in 'right','top':
        ax[a].spines[side].set_visible(False)
    ax[a].set_xlim(ticks[a][[0,-1]] + np.array([0,1]))
    ax[a].xaxis.set_major_locator(FixedLocator(ticks[a]))
    ax[a].xaxis.set_minor_locator(NullLocator())
    ax[a].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks[a]]))
ax['A'].legend(loc='lower left', frameon=False, fontsize=6)
ax['A'].set_ylabel('Power [dB]')
ax['A'].set_ylim([-60, -10])
ax['B'].set_xlim([0.4, 1.5])
ax['B'].set_ylim([-33, -10])

################### Panel C ###################
key = experiment_IDs[0][:6]
ax['C'].plot(data[key]['ym']['test'], data[key]['ym']['test'], 'k--', lw=2, markerfacecolor='w')
m = 'so'
markers = {ID[:6]: m[i] for i,ID in enumerate(experiment_IDs)}
lw, ms = 2, 6
for ID in experiment_IDs:
    n = 0
    key = ID[:6]
    ym = data[key]['ym']
    ym_pred = data[key]['ym_pred']
    ys_pred = data[key]['ys_pred']
    for cond in ym:
        for i in range(len(ym[cond])):
            ax['C'].plot(ym[cond][i] + np.zeros(2),
                         ym_pred[cond][i] + ys_pred[cond][i] * np.array([-1,1]),
                         color=cmap(n), linewidth=lw)
            ax['C'].plot(ym[cond][i], ym_pred[cond][i], markers[key], color=cmap(n), markersize=ms,
                     markerfacecolor='w', markeredgewidth=lw)
            n += 1
ax['C'].set_xlabel(r'Exact M [GW$\cdot$s$^2$]')
ax['C'].set_ylabel(r'Predicted M [GW$\cdot$s$^2$]')
ym = ym['test']
ax['C'].set_xlim(ym + np.diff(ym) * np.array([-1/7, 1/5]))
ax['C'].set_ylim(ym + np.diff(ym) * np.array([-1/10, 1/10]))
ax['C'].xaxis.set_major_locator(FixedLocator(ym))
ax['C'].xaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in ym]))
ax['C'].yaxis.set_major_locator(FixedLocator(ym))
ax['C'].yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in ym]))

################### Panels D, E and F ###################
key = experiment_IDs[1][:6]
plot_correlations(correlations[key]['R'],
                  correlations[key]['p'],
                  correlations[key]['R_ctrl'],
                  correlations[key]['p_ctrl'],
                  correlations[key]['edges'],
                  [np.concatenate(correlations[key]['idx'])],
                  sort_freq=[1.1],
                  ax=np.array([[ax['D'], ax['E'], ax['F']]]),
                  legend_bbox=[0.3, -0.05])
plot_correlations(correlations[key]['R'],
                  correlations[key]['p'],
                  None,
                  None,
                  correlations[key]['edges'],
                  [np.concatenate(correlations[key]['idx'])],
                  sort_freq=[5.75],
                  ax=np.array([[ax['E']]]))

ax['D'].set_ylabel('Frequency [Hz]')
ax['D'].set_xlabel('Filter #')
ax['E'].set_xlabel('Filter #')
ax['F'].set_xlabel('Correlation')
ax['D'].set_xticklabels([1, 32, 64])
ax['E'].set_xticklabels([1, 32, 64])

for lbl in 'DEF':
    ax[lbl].set_ylim(ticks['A'][[0,-1]] + np.array([0,1]))
    ax[lbl].yaxis.set_major_locator(FixedLocator(ticks['A']))
    ax[lbl].yaxis.set_minor_locator(NullLocator())
    ax[lbl].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks['A']]))

trans = mtransforms.ScaledTranslation(-0.45, -0.05, fig.dpi_scale_trans)
for lbl,a in ax.items():
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.text(0.0, 1.0, lbl, transform=a.transAxes + trans, fontsize=10, va='bottom')

fig.tight_layout(pad=0)
pdf_file = f'variable_inertia_{experiment_IDs[0][:6]}_{experiment_IDs[1][:6]}.pdf'
fig.savefig(pdf_file)